<a href="https://colab.research.google.com/github/linyuehzzz/5523_project/blob/main/sgd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Stochastic Gradient Descent for Logistic Regression**
This code implements and tests the SGD algorithm for logistic regression
in different scenarios.  
Yue Lin (lin.3326 at osu.edu)  
Created: 11/12/2020

#### **Set up libraries**

In [171]:
import numpy as np
import random
import pandas as pd
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler

#### **Project**

##### Projection function for hypercube

In [172]:
def cube_prj(sample):
  '''
  input: a sample with d dimension (1-D array or list)
  onput: the euclidean projection of sample
  '''
  return [np.sign(i) * min(np.abs(i), 1) for i in sample]

##### Projection function for unit ball

In [173]:
def ball_prj(sample):
    '''
    input: a sample with d dimension (1-D array or list)
    onput: the euclidean projection of sample
    '''
    ratio = 1 / np.linalg.norm(sample)
    return [i * ratio for i in sample]

##### Project data

In [174]:
def prj_data(x, y, prj_code):
    '''
    This function is for conduct projection on data array
    x: n*d array (n is sample#, d is dimension)
    y: 1-d array with label of -1 or +1
    prj_code: type of projection, 0 for cube, 1 for ball
    return:
        prj_x: projected x 
        y: same as input
    '''
    if prj_code == 0:
        prj_x = np.apply_along_axis(cube_prj, 1, x)
    elif prj_code == 1:
        prj_x = np.apply_along_axis(ball_prj, 1, x)
    else:
        print("Please input correct code for projection type: 0 for cube, 1 for ball")
      
    b = np.ones((prj_x.shape[0], 1))
    prj_x = np.append(prj_x, b, axis=1)
    return prj_x, y

##### Project gradient

In [175]:
def prj_grad(g, prj_code):
    '''
    This function is for conduct projection on gradients
    g: 1-d array (d is dimension)
    prj_code: type of projection, 0 for cube, 1 for ball
    return:
        prj_g: projected gradient
    '''
    if prj_code == 0:
        prj_g = cube_prj(g)
    elif prj_code == 1:
        prj_g = ball_prj(g)
    else:
        print("Please input correct code for projection type: 0 for cube, 1 for ball")
    return prj_g

#### **Prepare data**

In [176]:
def gen_data(sig, n):
    '''
    The function is to generate data for training and testing
    The feature array is 4 dimension array. 
     + Each feature follows the Normal distribution(mu,sig)
     + with probability 1/2, the y =1 , 
         generate the correspoinding feature vector from N(mu,sig),mu is [1/4,1/4,1/4,1/4],sig is set as you need.
    sig: the sigma of  gussian vector
    n: the sample number
    
    Return:
     x: n*d_dimension array
     y: 1-d dimension array with -1 and +1
    '''
    d_dimension = 4
    y = np.random.choice([-1, 1], p = [0.5, 0.5], size = n)
    x = np.array([])
    for i in range(n):
        if y[i] == -1:
            mu = -(1 / 4)
            negvec = np.random.normal(mu, sig, d_dimension)
            x = np.concatenate([x, negvec], axis=0)
        else:
            mu = (1/4)
            posvec = np.random.normal(mu, sig, d_dimension)
            x = np.concatenate([x,posvec], axis=0)
    x = np.reshape(x, (n, d_dimension))
    return x, y

#### **Train**
https://machinelearningmastery.com/implement-logistic-regression-stochastic-gradient-descent-scratch-python/

##### Predict using logistic regression

In [177]:
# Make a prediction with coefficients
def pred(X, w):
  yhat = 0.
  for i in range(X.shape[0]):
    yhat += w[i] * X[i]
  yhat = 1.0 / (1.0 + np.exp(-yhat))
  if yhat < 0.5:
    yhat = -1
  else:
    yhat = 1 
  return yhat

##### Estimate logistic loss

In [178]:
def log_loss(X, y, w):
  return np.log(1 + np.exp(-y * np.dot(w.T, X)))

##### Estimate classification error

In [179]:
def err(yhat, y):
  if yhat == y:
    return 0
  else:
    return 1

##### Estimate weight vector using SGD

In [180]:
def train_sgd(train_x, train_y, test_x, test_y, l_rate, n_epoch, bs, prj_code):
  w = np.random.uniform(-1, 1, (train_x.shape[1]))
  print(w)
  risk_all = np.zeros(n_epoch)
  cls_err_all = np.zeros(n_epoch)

  for epoch in range(n_epoch):
    risk = cls_err = grad = 0.
    for idx in range(epoch * bs, (epoch + 1) * bs):
      # Read data
      X = train_x[idx]
      y = train_y[idx]
      # Calculate gradient
      g = (-y * X * np.exp(-y * np.dot(w.T, X)) / (1 + np.exp(-y * np.dot(w.T, X))))
      for i in range(train_x.shape[1]):
        g[i] = g[i] * X[i]
      grad += g

    # Project gradient
    grad = prj_grad(grad / bs, prj_code)
    # Backward propagation
    for i in range(train_x.shape[1]):
      w[i] = w[i] + l_rate * grad[i]
    
    # Evaluate
    for idx in range(test_x.shape[0]):
      # Read data
      X = test_x[idx]
      y = test_y[idx]
      # Predict
      yhat = pred(X, w)
      # Evaluate
      risk += log_loss(X, y, w) / test_x.shape[0]
      cls_err += err(yhat, y) / test_x.shape[0]
    
    np.append(risk_all, risk, axis=None)
    np.append(cls_err_all, cls_err, axis=None)
    print('>epoch=%d, lrate=%.3f, risk=%.3f, classification error=%.3f' % (epoch, l_rate, risk, cls_err))
  
  # Report risk
  risk_ave = np.average(risk_all)
  risk_min = np.amin(risk_all)
  risk_var = np.var(risk_all)
  exp_excess_risk = risk_ave - risk_min
  # Report classification error
  cls_err_ave = np.average(cls_err_all)
  cls_err_var = np.var(cls_err_all)
  return [w, risk_ave, risk_min, risk_var, exp_excess_risk, cls_err_ave, cls_err_var]

#### **Wrapper**

In [181]:
# Fixed hyperparameters
n_epoch = 30    # training epochs
test_n = 400    # size of test set

# Unfixed hyperparameters
prj_code = 1    # code for two scenario: 0 for cube, 1 for ball
l_rate = 0.001  # learning rate
train_bs = 50   # batch size for each training epoch
sigma = 0.1     # variance of Gaussian distribution

# Generate training data
train_x, train_y = gen_data(sigma, train_bs * n_epoch)
train_px, train_py = prj_data(train_x, train_y, prj_code)

# Generate test data
test_x, test_y = gen_data(sigma, test_n)
test_px, test_py = prj_data(test_x, test_y, prj_code)

# Train
output = train_sgd(train_px, train_py, test_px, test_py, l_rate, n_epoch, train_bs, prj_code)


[-0.6811076   0.26494992  0.35414407  0.20112446 -0.09472023]
>epoch=0, lrate=0.001, risk=0.665, classification error=0.375
>epoch=1, lrate=0.001, risk=0.665, classification error=0.375
>epoch=2, lrate=0.001, risk=0.664, classification error=0.373
>epoch=3, lrate=0.001, risk=0.665, classification error=0.373
>epoch=4, lrate=0.001, risk=0.664, classification error=0.373
>epoch=5, lrate=0.001, risk=0.665, classification error=0.373
>epoch=6, lrate=0.001, risk=0.665, classification error=0.378
>epoch=7, lrate=0.001, risk=0.665, classification error=0.375
>epoch=8, lrate=0.001, risk=0.665, classification error=0.378
>epoch=9, lrate=0.001, risk=0.665, classification error=0.375
>epoch=10, lrate=0.001, risk=0.664, classification error=0.373
>epoch=11, lrate=0.001, risk=0.665, classification error=0.375
>epoch=12, lrate=0.001, risk=0.665, classification error=0.378
>epoch=13, lrate=0.001, risk=0.665, classification error=0.375
>epoch=14, lrate=0.001, risk=0.664, classification error=0.373
>ep

NameError: ignored